In [ ]:
%load_ext autoreload
%autoreload 2

import glob
from fibsem.segmentation.model import SegmentationModel
import tifffile as tf
import matplotlib.pyplot as plt


from random import shuffle

import numpy as np
from fibsem.detection.utils import Feature, FeatureType, DetectionResult
from autoscript_sdb_microscope_client.structures import AdornedImage

from fibsem.structures import Point
from fibsem.imaging import masks
from fibsem.detection import detection
import skimage

from pathlib import Path
from dataclasses import dataclass

from fibsem import conversions


from pprint import pprint


from fibsem.segmentation.model import load_model

## Detection Goals


1. Detect Needle Tip
2. Detect Lamella Centre
3. Detect Lamella Edges (Right / Left, Up / Down)

account for multiple lamellas
mask centre?

In [ ]:


# data

from fibsem import config



filenames = glob.glob(r"C:\Users\Admin\Github\autoliftout\liftout\log\dm-E10-16Nov22-2022-12-06.11-07-08AM/**/*landing_needle_start_position_0_ib.tif")
print(len(filenames))

# model
checkpoint = r"C:\Users\Admin\Github\fibsem\fibsem\segmentation\models\model4.pt"
model = load_model(checkpoint, encoder="resnet34")





def to_bounding_box(contour):
    # convert a contour to bounding box (xc, yc, w, h)
    # ref : https://muthu.co/draw-bounding-box-around-contours-skimage/

    xmin, xmax = np.min(contour[:, 1]), np.max(contour[:, 1])
    ymin, ymax = np.min(contour[:, 0]), np.max(contour[:, 0])

    w = (xmax - xmin)
    h = (ymax - ymin)
    xc = xmin + w // 2
    yc = ymin + h // 2
    
    return [xc, yc, w, h]
    




# THINGS TO TRY:
# masking centre area for lamella
# using contours to extract individual lamellas -> pick centre

mask_radius = 400

shuffle(filenames)
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    # inference
    mask = model.inference(img)

    # detect features
    features = [Feature(FeatureType.LamellaRightEdge), 
                    Feature(FeatureType.LandingPost)]
    det = detection.locate_shift_between_features_v2(img, model, features=features, pixelsize=10e-9, mask_radius=mask_radius)

    lamella_mask = masks.apply_circular_mask(img, radius=mask_radius)

    plt.imshow(lamella_mask)
    plt.show()

    # plot
    detection.plot_det_result_v2(det, save=False)

    if i == 10:
        break


In [ ]:


# mask helper


# centre circle
# left/right half
# top/bottom half


from fibsem.imaging import masks
import numpy as np


arr = np.zeros(shape=(720, 1200 ))

circ_mask = masks.create_circle_mask(arr.shape, radius=128)

bl_mask = masks.create_area_mask(arr, left=True, lower=True)
ul_mask = masks.create_area_mask(arr, left=True, upper=True)
br_mask = masks.create_area_mask(arr, left=True, lower=True)
ur_mask = masks.create_area_mask(arr, right=True, upper=True)
mask = masks.create_area_mask(arr, left=True)


fig, ax = plt.subplots(1, 5, figsize=(15, 7))
ax[0].imshow(bl_mask)
ax[1].imshow(ul_mask)
ax[2].imshow(br_mask)
ax[3].imshow(ur_mask)
ax[4].imshow(mask)
plt.show()



In [ ]:
# contour stuff
    # options for getting individual lamella from multiple: contours, masking 
    # bboxes = []
    # contours = skimage.measure.find_contours(lamella_mask[:, :, 0].astype(np.uint8), 0.8)
    # for contour in contours:
    #     bboxes.append(to_bounding_box(contour))
        # for contour in contours:
    #     ax[1].plot(contour[:, 1], contour[:, 0], color="white", linewidth=1)
    

In [ ]:
## vertical mask for eucentric correction

# autoreload 2
%load_ext autoreload
%autoreload 2

from fibsem.imaging import masks
import numpy as np
import matplotlib.pyplot as plt

# mask only the central vertical area
arr = np.zeros(shape=(1024, 1536 ))

mask = masks.create_vertical_mask(arr)

fig, ax = plt.subplots(1, 1, figsize=(15, 7))
ax.imshow(mask)
plt.show()



##### Centre Left / Right Detection

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem.detection import detection
import numpy as np
import matplotlib.pyplot as plt
from fibsem.structures import Point



# # mask the middle 100px of image
# rgb = np.zeros(shape=(1024, 1536, 3))
# rgb[500:600, 500:600, 0] = 255
# rgb[500:600, 500:600, 1] = 0
# rgb[500:600, 500:600, 2] = 0

# rgb = np.zeros(shape=(1024, 1536))
# rgb[500:600, 500:600] = 1


rgb = np.zeros(shape=(1024, 1536, 3))

arr = rgb
centre = detection.get_mask_point(arr, "centre", "centre")
left = detection.get_mask_point(arr, "left", "centre")
right = detection.get_mask_point(arr, "right", "centre")
upper = detection.get_mask_point(arr, "centre", "upper")
lower = detection.get_mask_point(arr, "centre", "lower")
top_left = detection.get_mask_point(arr, "left", "upper")
top_right = detection.get_mask_point(arr, "right", "upper")
bottom_left = detection.get_mask_point(arr, "left", "lower")
bottom_right = detection.get_mask_point(arr, "right", "lower")


# plot all points and mask with labels and legends
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
ax.imshow(arr, cmap="gray")
ax.scatter(centre.x, centre.y, color="red", label="centre")
ax.scatter(left.x, left.y, color="green", label="left")
ax.scatter(right.x, right.y, color="blue", label="right")
ax.scatter(upper.x, upper.y, color="orange", label="upper")
ax.scatter(lower.x, lower.y, color="purple", label="lower")
ax.scatter(top_left.x, top_left.y, color="black", label="top left")
ax.scatter(top_right.x, top_right.y, color="grey", label="top right")
ax.scatter(bottom_left.x, bottom_left.y, color="pink", label="bottom left")
ax.scatter(bottom_right.x, bottom_right.y, color="yellow", label="bottom right")
ax.legend()
plt.show()

# TODO: detection is very inefficient, need to find a way to clean it up